#### 모델을 학습시키기 전에 의사결정

1. 어떤 loss를 쓸 것인지
2. learning rate은 몇으로 할 것인지
3. optimizer는 무엇으로 할 것인지
4. 학습 횟수는 몇 번으로 할 것인지

#### 기본루틴
- *optimizer.zero_grad()*  
- *pred = model(x)*  
- *loss = criterion(pred, x_labels)*  
- *loss.backward()*  
- *optimizer.step()*  

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

In [4]:
class MyModel(nn.Module):
    
    def __init__(self, X_dim, y_dim):
        super(MyModel, self).__init__()
        layer1 = nn.Linear(X_dim, 128)
        activation1 = nn.ReLU()
        layer2 = nn.Linear(128, y_dim)
        self.module = nn.Sequential(
            layer1,
            activation1,
            layer2
        )
        
    def forward(self, x):
        out = self.module(x)
        result = F.softmax(out, dim=1)
        return result        

In [62]:
# 준비재료
criterion = nn.CrossEntropyLoss()
learning_rate = 1e-5
optimizer = optim.SGD(model.paraeters(), lr=learning_rate)
num_epochs = 2
num_batches = len(train_loader)

for epoch in range(num_epochs):
	for i, data in enumerate(train_loader):
		x, x_labels = data # x.size() = [batch, channel, x, y]
		# init grad
		optimizer.zero_grad() # step과 zero_grad는 쌍을 이루는 것이라고 생각하면 됨
		# forward
		pred = model(x)
		# calculate loss
		loss = criterion(pred, x_labels)
		# backpropagation
		loss.backward()
		# weight update
		optimizer.step()
		# 학습과정 출력
		running_loss += loss.item()
		if (i+1)%2000 == 0: # print every 2000 mini-batches
			print("epoch: {}/{} | step: {}/{} | loss: {:.4f}".format(epoch, num_epochs, i+1, num_batches, running_loss/2000))
			running_loss = 0.0

print("finish Training!")

NameError: name 'model' is not defined

## 텐서 다루기 https://wikidocs.net/52460

- 자연어 처리는 보통 (batch size, 문장 길이, 단어 벡터의 차원)이라는 3차원 텐서를 사용함
    - 문장 샘플들, 문장 샘플들의 길이(단어 수), 해당 문장에 대한 차원(밀집 벡터)

In [77]:
# 3차원 감각 익히기
t = np.array([[[0, 1, 2, 4], [3, 4, 5, 4], [2, 5, 4, 3]],
            [[6, 7, 8, 4], [9, 10, 11, 4], [8, 13, 24, 5]]])
ft = torch.FloatTensor(t)

In [16]:
print(ft.shape)  # 샘플의 최종 묶음(문장)은 두개, 세개 단어씩, 그리고 각 문장마다 표현할 밀집 벡터. 가장 바깥쪽 괄호부터 하나씩 진입해서 보면 눈에 보인다

torch.Size([2, 2, 3])


4) 뷰

In [3]:
# 4) 뷰 (https://wikidocs.net/52846)
t = np.array([[[0, 1, 2],
               [3, 4, 5]],
              [[6, 7, 8],
               [9, 10, 11]]])
ft = torch.FloatTensor(t)

In [10]:
print(ft)
print(ft.shape)

tensor([[[ 0.,  1.,  2.],
         [ 3.,  4.,  5.]],

        [[ 6.,  7.,  8.],
         [ 9., 10., 11.]]])
torch.Size([2, 2, 3])


In [11]:
# 3차원 to 2차원
print(ft.view([-1, 3])) # ft라는 텐서를 (?, 3)의 크기로 변경
print(ft.view([-1, 3]).shape)  # -1은 파이토치에게 맡기는 것. 결과적으로 (4, 3)의 크기를 가지는 텐서로 처리됨

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.]])
torch.Size([4, 3])


In [17]:
# 3차원 텐서 크기만 변경(샘플 개수)
print(ft.view([-1, 1, 3]))
print(ft.view([-1, 1, 3]).shape)

tensor([[[ 0.,  1.,  2.]],

        [[ 3.,  4.,  5.]],

        [[ 6.,  7.,  8.]],

        [[ 9., 10., 11.]]])
torch.Size([4, 1, 3])


5) 스퀴즈 - 1인 차원을 제거

In [39]:
# 5) 스퀴즈 - 1인 차원을 제거함
ft = torch.FloatTensor([[0], [1], [2]])  # 2차원
print(ft)
print(ft.shape)

tensor([[0.],
        [1.],
        [2.]])
torch.Size([3, 1])


In [23]:
print(ft.squeeze())   # 1인 차원이 제거되어 1차원 벡터로 변환됨
print(ft.squeeze().shape)

tensor([0., 1., 2.])
torch.Size([3])


6) 언스퀴즈 - 특정 위치에 1인 차원을 추가

In [30]:
# 특정 위치에 1인 차원을 추가함
ft = torch.Tensor([0, 1, 2])  # 1차원
print(ft.shape)

torch.Size([3])


In [31]:
print(ft.unsqueeze(0)) # 인덱스가 0부터 시작하므로 0은 첫번째 차원을 의미
print(ft.unsqueeze(0).shape)

tensor([[0., 1., 2.]])
torch.Size([1, 3])


In [43]:
# 동일한 처리를 view로
print(ft.view(1, -1))  # 첫번째 차원을 1로 (x축(행)이 1개)
print(ft.view(1, -1).shape)
print(ft.view(3, -1))  # 첫번째 차원을 3으로 (x축(행)이 3개)
print(ft.view(3, -1).shape)

tensor([[0., 1., 2.]])
torch.Size([1, 3])
tensor([[0.],
        [1.],
        [2.]])
torch.Size([3, 1])


In [ ]:
# Question
# 1) ft.unsqueeze(1)
# 2) ft.unsquessze(-1)

## 단어의 표현 방법 https://wikidocs.net/60852


### 원핫 인코딩 한계
 1. 단어 개수가 늘어날수록 벡터 공간도 함께 늘어남(저장 공간 측면에서 매우 비효율적)
 2. 단어의 유사도를 표현하지 못함

### 희소 표현(희소 벡터)
 - 벡터 또는 행렬(matrix)의 값이 대부분 0으로 표현되는 방법을 희소 표현(sparse representation)이라고 함). 원-핫 벡터는 희소 벡터이다(원핫 인코딩의 한계를 그대로 가짐)
 
 Ex) 강아지 = [ 0 0 0 0 1 0 0 0 0 0 0 0 ... 중략 ... 0] # 이 때 1 뒤의 0의 수는 9995개. 차원은 10,000 (단어가 10,000개라고 가정 시)
 
### 밀집 표현(밀집 벡터)
 - 사용자가 밀집 표현의 차원을 128로 설정한다면, 모든 단어의 벡터 표현의 차원은 128로 바뀌면서 모든 값이 실수가 된다.
 
 Ex) 강아지 = [0.2 1.8 1.1 -2.1 1.1 2.8 ... 중략 ...] # 이 벡터의 차원은 128
 
### 워드 임베딩
 - 단어를 밀집 벡터(dense vector)의 형태로 표현하는 방법을 워드 임베딩이라고 함. 그리고 이 밀집 벡터를 워드임베딩 과정을 통해 나온 결과라고 하여 임베딩 벡터라고도 한다.
 
 - 워드 임베딩 방법론으로는 LSA, Word2Vec, FastText, Glove 등이 있음 
 - Pytorch 에서 제공하는 도구인 nn.embedding()는 앞서 언급한 방법들을 사용하지는 않지만, 단어를 랜덤한 값을 가지는 밀집 벡터로 변환한 뒤에, 인공 신경망의 가중치를 학습하는 것과 같은 방식으로 단어 벡터를 학습하는 방법을 사용함
 
### 워드투벡터(Word2Vec)
 - 단어 간 유사도를 반영할 수 있도록 단어의 의미를 벡터화 하는 방법
 - 분산 표현 : 단어의 '의미'를 다차원 공간에 벡터화하는 방법
 - 이렇게 분산 표현을 이용하여 단어의 유사도를 벡터화하는 작업은 워드 임베딩(embedding) 작업에 속하기 때문에 이렇게 표현된 벡터 또한 임베딩 벡터(embedding vector)라고 하며, 저차원을 가지므로 바로 앞의 챕터에서 배운 밀집 벡터(dense vector)에도 속함
 
### 분산표현
 - 요약하면 희소 표현이 고차원에 각 차원이 분리된 표현 방법이었다면, 분산 표현은 저차원에 단어의 의미를 여러 차원에다가 분산하여 표현한다. 이런 표현 방법을 사용하면 단어 간 유사도를 계산할 수 있다.

 - 이를 위한 학습 방법으로는 NNLM, RNNLM 등이 있으나 요즘에는 해당 방법들의 속도를 대폭 개선시킨 Word2Vec가 많이 쓰이고 있다.

---------------------------------------------

### PyTorch의 nn.Embedding()
- 파이토치에서는 임베딩 벡터를 사용하는 방법이 크게 두 가지가 있다. 
    - 임베딩 층(embedding layer)을 만들어 훈련 데이터로부터 처음부터 임베딩 벡터를 학습하는 방법
    - 미리 사전에 훈련된 임베딩 벡터(pre-trained word embedding)들을 가져와 사용하는 방법
    
### 임베딩 층은 룩업 테이블이다.
- 임베딩 층의 입력으로 사용하기 위해서 입력 시퀀스의 각 단어들은 모두 정수 인코딩이 되어있어야 한다

어떤 단어 → 단어에 부여된 고유한 정수값 → 임베딩 층 통과 → 밀집 벡터

1) 임베딩 층은 입력 정수에 대해 밀집 벡터(dense vector)로 매핑

2) 이 밀집 벡터는 인공 신경망의 학습 과정에서 가중치가 학습되는 것과 같은 방식으로 훈련됨. 

3) 훈련 과정에서 단어는 모델이 풀고자하는 작업에 맞는 값으로 업데이트 된다. 

이 밀집 벡터를 임베딩 벡터라고 부른다.
    

정수를 밀집 벡터 또는 임베딩 벡터로 맵핑한다는 것은 어떤 의미일까요? 특정 단어와 맵핑되는 정수를 인덱스로 가지는 테이블로부터 임베딩 벡터 값을 가져오는 룩업 테이블이라고 볼 수 있습니다. 그리고 이 테이블은 단어 집합의 크기만큼의 행을 가지므로 모든 단어는 고유한 임베딩 벡터를 가집니다.

룩업테이블에 있는 단어의 임베딩 벡터들은 역전파 과정에서 값이 학습된다.

In [4]:
import torch
train_data = 'you need to know how to code'
word_set = set(train_data.split()) # 중복을 제거한 단어들의 집합인 단어 집합 생성.
vocab = {word: i+2 for i, word in enumerate(word_set)}  # 단어 집합의 각 단어에 고유한 정수 매핑.
vocab['<unk>'] = 0
vocab['<pad>'] = 1
print(vocab)

{'to': 2, 'code': 3, 'need': 4, 'know': 5, 'you': 6, 'how': 7, '<unk>': 0, '<pad>': 1}


In [5]:
# 단어 집합의 크기만큼의 행을 가지는 테이블 생성. (임베딩 벡터의 차원은 3이라 가정)
embedding_table = torch.FloatTensor([
                               [ 0.0,  0.0,  0.0],  # 0
                               [ 0.0,  0.0,  0.0],  # 1
                               [ 0.2,  0.9,  0.3],  # 2
                               [ 0.1,  0.5,  0.7],  # 3
                               [ 0.2,  0.1,  0.8],  # 4
                               [ 0.4,  0.1,  0.1],  # 5
                               [ 0.1,  0.8,  0.9],  # 6
                               [ 0.6,  0.1,  0.1]])  # 7

In [8]:
# 임의의 샘플 문장
sample = 'you need to run'.split()
idxes=[]
# 각 단어를 정수로 변환
for word in sample:
  try:
    idxes.append(vocab[word])
  except KeyError: # 단어 집합에 없는 단어일 경우 <unk>로 대체된다.
    idxes.append(vocab['<unk>'])
idxes = torch.LongTensor(idxes) # tensor([6, 4, 2, 0])

# 룩업 테이블
lookup_result = embedding_table[idxes, :] # 각 정수를 인덱스로 임베딩 테이블에서 값을 가져온다.
print(lookup_result)

tensor([[0.1000, 0.8000, 0.9000],
        [0.2000, 0.1000, 0.8000],
        [0.2000, 0.9000, 0.3000],
        [0.0000, 0.0000, 0.0000]])


### 임베딩 층 사용하기

nn.Embedding은 크게 두 가지 인자를 받는데 각각 num_embeddings과 embedding_dim입니다.

 - num_embeddings : 임베딩을 할 단어들의 개수. 다시 말해 단어 집합의 크기.
 - embedding_dim : 임베딩 할 벡터의 차원. 사용자가 정해주는 하이퍼파라미터이다.
 - padding_idx : 선택적으로 사용하는 인자. 패딩을 위한 토큰의 인덱스를 알려준다.

In [ ]:
train_data = 'you need to know how to code'
word_set = set(train_data.split()) # 중복을 제거한 단어들의 집합인 단어 집합 생성.
vocab = {tkn: i+2 for i, tkn in enumerate(word_set)}  # 단어 집합의 각 단어에 고유한 정수 맵핑.
vocab['<unk>'] = 0
vocab['<pad>'] = 1

In [11]:
import torch.nn as nn

embedding_layer = nn.Embedding(num_embeddings = len(vocab), 
                               embedding_dim = 3,
                               padding_idx = 1)

In [14]:
#print(embedding_layer)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 2.8888,  1.5721,  0.5345],
        [ 0.0000,  0.0000,  0.0000],
        [-0.2195,  1.0877, -0.9658],
        [-0.9405,  0.2130,  1.4181],
        [ 0.8394,  0.2810,  0.8668],
        [ 0.2448, -0.5023,  0.9176],
        [-0.8760,  0.8622,  0.7863],
        [-0.4443,  0.6083, -0.7428]], requires_grad=True)


### 사전 훈련된 워드 임베딩(Pretrained Word Embedding) : https://wikidocs.net/64904
 - nn.Embedding()을 사용하는 것보다 다른 텍스트 데이터로 사전 훈련되어 있는 임베딩 벡터를 불러오는 것이 나은 선택일 수 있다.
 - 훈련 데이터가 적다면 파이토치의 nn.Embedding()으로 해당 문제에 충분히 특화된 임베딩 벡터를 만들어내는 것이 쉽지 않다. 이 경우, 해당 문제에 특화된 것은 아니지만 보다 일반적이고 보다 많은 훈련 데이터로 이미 Word2Vec이나 GloVe 등으로 학습되어져 있는 임베딩 벡터들을 사용하는 것이 성능의 개선을 가져올 수 있다.


In [16]:
from torchtext import data, datasets

ModuleNotFoundError: No module named 'torchtext'

### 임베딩 실습

In [18]:
src_vocab_size = 5 # 단어 사전 크기
d_model = 512  # 단어 임베딩 크기 지정(하이퍼 파라미터)
src_len = 5 # length of source?
n_layers = 6  # number of Encoder of Decoder Layer
n_heads = 8  # number of heads in Multi-Head Attention

nn.Embedding(src_vocab_size, d_model) # 128 크기의 단어수 만큼 임베딩 벡터 공간을 만듬
# nn.Embedding.from_pretrained(get_sinusoid_encoding_table(src_len+1, d_model),freeze=True) # Positional Encoding
nn.ModuleList([EncoderLayer() for _ in range(n_layers)]) # 인코더 레이어 생성. ModuleList를 활용하여 레이어 층만큼 반복 생성. 총 6개 생성


Embedding(5, 128)

In [ ]:
# 잘 이해 x
def get_sinusoid_encoding_table(n_position, d_model):  # src_len+1, 512


In [19]:
# 인코더층은 Self-Attention과 Feed Forward로 구성됨. 이를 선언하고 전이시키는 역할
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()
        
    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn

SyntaxError: invalid syntax (<ipython-input-19-bfba0f233d71>, line 5)

In [47]:
d_model = 512  # 단어 임베딩 크기 지정(하이퍼 파라미터)
n_heads = 8  # number of heads in Multi-Head Attention
d_k = d_v = 64  # dimension of K(=Q), V

W_Q = nn.Linear(d_model, d_k * n_heads)  # input : 512 , output : 64 * 8(=512)
W_Q

Linear(in_features=512, out_features=512, bias=True)

In [22]:
nn.Linear(n_heads * d_v, d_model)

Linear(in_features=512, out_features=512, bias=True)

In [23]:
self.layer_norm = nn.LayerNorm(d_model) # ??

NameError: name 'self' is not defined

In [26]:
sentences = ['ich mochte ein bier P', 'S i want a beer', 'i want a beer E']

In [28]:
src_vocab = {'P': 0, 'ich': 1, 'mochte': 2, 'ein': 3, 'bier': 4}
src_vocab

{'P': 0, 'bier': 4, 'ein': 3, 'ich': 1, 'mochte': 2}

In [30]:
tgt_vocab = {'P': 0, 'i': 1, 'want': 2, 'a': 3, 'beer': 4, 'S': 5, 'E': 6}
tgt_vocab

{'E': 6, 'P': 0, 'S': 5, 'a': 3, 'beer': 4, 'i': 1, 'want': 2}

In [31]:
number_dict = {i: w for i, w in enumerate(tgt_vocab)}
number_dict

{0: 'P', 1: 'i', 2: 'want', 3: 'a', 4: 'beer', 5: 'S', 6: 'E'}

In [32]:
input_batch = [[src_vocab[n] for n in sentences[0].split()]]
output_batch = [[tgt_vocab[n] for n in sentences[1].split()]]
target_batch = [[tgt_vocab[n] for n in sentences[2].split()]]

In [36]:
print(input_batch)
print(output_batch)
print(target_batch)

[[1, 2, 3, 4, 0]]
[[5, 1, 2, 3, 4]]
[[1, 2, 3, 4, 6]]


In [39]:
print(torch.LongTensor(input_batch))
print(torch.LongTensor(output_batch))
print(torch.LongTensor(target_batch))

tensor([[1, 2, 3, 4, 0]])
tensor([[5, 1, 2, 3, 4]])
tensor([[1, 2, 3, 4, 6]])


In [52]:
# 이제 멀티어센션에서 실제 값 연산 하는 과정인데 구조 복잡해졌음. 여기만 넘으면 된다

# 호출부 이전 : enc_inputs, dec_inputs, target_batch = make_batch(sentences) # 임베딩된 데이터 
# 호출부 : enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V (임베딩된 데이터 전달)

# q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
q_s = W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
#연산인거같은데?? view는 머임??

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x5 and 512x512)

#### 아래 확인 후 돌아올 것

In [49]:
src_vocab = {'P': 0, 'ich': 1, 'mochte': 2, 'ein': 3, 'bier': 4}
src_vocab_size = len(src_vocab)
print(src_vocab_size)
tgt_vocab = {'P': 0, 'i': 1, 'want': 2, 'a': 3, 'beer': 4, 'S': 5, 'E': 6}
number_dict = {i: w for i, w in enumerate(tgt_vocab)}
tgt_vocab_size = len(tgt_vocab)
print(tgt_vocab_size)
d_model = 512
print(d_model)

5
7
512


In [50]:
def make_batch(sentences):
    input_batch = [[src_vocab[n] for n in sentences[0].split()]] # input 문장을 단어로 분리 후 워드임베딩화
    output_batch = [[tgt_vocab[n] for n in sentences[1].split()]] # output 문장을 단어로 분리 후 워드임베딩화
    target_batch = [[tgt_vocab[n] for n in sentences[2].split()]] 
    return torch.LongTensor(input_batch), torch.LongTensor(output_batch), torch.LongTensor(target_batch)

In [51]:
sentences = ['ich mochte ein bier P', 'S i want a beer', 'i want a beer E']
enc_inputs, dec_inputs, target_batch = make_batch(sentences)

enc_inputs # Q, K, V

tensor([[1, 2, 3, 4, 0]])

In [52]:
#enc_inputs -> 임베딩 해야지
src_emb = nn.Embedding(src_vocab_size, d_model)
src_emb

Embedding(5, 512)

In [53]:
enc_outputs = src_emb(enc_inputs)  # 5개의 단어를 512 차원으로 변환
enc_outputs

tensor([[[-0.6487, -0.4674,  0.1753,  ..., -0.5424, -0.7864, -0.4046],
         [-0.3876, -0.3510, -0.9609,  ...,  0.2835, -1.0932, -0.6259],
         [-1.1307, -1.9039, -0.2147,  ...,  1.4949,  0.1062, -1.0218],
         [-0.4046, -3.1268, -0.1270,  ..., -0.1050,  0.0824, -0.0577],
         [ 0.9097, -0.8360, -1.4804,  ...,  0.5880,  0.3441, -0.3348]]],
       grad_fn=<EmbeddingBackward>)

In [54]:
d_model = 512  # 단어 임베딩 크기 지정(하이퍼 파라미터)
n_heads = 8  # number of heads in Multi-Head Attention
d_k = d_v = 64  # dimension of K(=Q), V


W_Q = nn.Linear(d_model, d_k * n_heads)  # 가중치 행렬
Q = enc_outputs
residual, batch_size = Q, Q.size(0) # ??
print(batch_size)
#W_Q(Q)
#W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)

1


In [ ]:
# https://catsirup.github.io/ai/2020/04/09/transformer-code.html  (참조필요)
# http://incredible.ai/nlp/2020/02/29/Transformer/#261-q-k-and-v